In [7]:
using Pkg
Pkg.add("Optim")
Pkg.add("Roots")
Pkg.add("NPZ")
Pkg.add("Interpolations")
Pkg.add("Distributed")
Pkg.add("Distributions")

using Optim
using Roots
using NPZ
using Distributed
using Random
using Distributions

In [2]:
gamma = 2.0
rho = 1.00001
fraction = 0.0
Delta = 300
dataname = "local"

ell_ex = 1/(gamma-1)
symmetric_returns    = 0
state_dependent_xi   = 0
optimize_over_ell    = 0
compute_irfs         = 0                    # need to start julia with "-p 5"

alpha_z_tilde_ex = 0.0#-.005

0.0

In [3]:
if compute_irfs == 1
    @everywhere include("newsets_utils_rho0.jl")
elseif compute_irfs ==0
    include("newsets_utils_rho0.jl")
end

println("=============================================================")
if symmetric_returns == 1
    println(" Economy with two capital stocks: SYMMETRIC RETURNS          ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_sym_HS.npz" : "model_sym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS.npz" : "model_sym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS2.npz" : "model_sym_HSHS2_p.npz";
    end
elseif symmetric_returns == 0
    println(" Economy with two capital stocks: ASYMMETRIC RETURNS         ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_asym_HS.npz" : "model_asym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS.npz" : "model_asym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS2.npz" : "model_asym_HSHS2_p.npz";
    end
end

filename_ell = "./output/"*dataname*"/"

 Economy with two capital stocks: ASYMMETRIC RETURNS         
 No tilting (xi is NOT state dependent)                      


"./output/local/"

In [4]:
#==============================================================================#
#  PARAMETERS
#==============================================================================#
delta = .002;

# (0) Single capital economy
alpha_c_hat = .484;
beta_hat = 1.0;
sigma_c = [.477, .0];

#===========================  CALIBRATION  ====================================#
# consumption_investment = 3.1;
#A_1cap, phi_1cap, alpha_k_hat, investment_capital = calibration2(15.,
#                                             consumption_investment,
#                                             alpha_c_hat, delta, sigma_c)
# A_1cap, phi_1cap, alpha_k_hat = calibration3(investment_capital,
#                                   consumption_investment,
#                                   alpha_c_hat, delta, sigma_c)
#

A_1cap = .05
phi_1cap = 28.
investment_capital, consumption_investment, alpha_k_hat = calibration3(phi_1cap,
                                            A_1cap, delta, alpha_c_hat, sigma_c)

println("  Calibrated values: A:", A_1cap,
        "  phi_1cap: ", phi_1cap,
        "  alpha_k : ", alpha_k_hat,
        "  C/I : ", consumption_investment,
        "  I/K : ", investment_capital)
println("=============================================================")
#==============================================================================#

# (1) Baseline model
alpha_z_hat = .0;
kappa_hat = .014;
zbar = alpha_z_hat/kappa_hat;
sigma_z_1cap = [.011, .025];

sigma_z =  [.011*sqrt(.5)   , .011*sqrt(.5)   , .025];


if symmetric_returns == 1

    beta2_hat = beta1_hat = 0.5;

    # (2) Technology
    phi2 = phi1 = phi_1cap;
    A2 = A1 = A_1cap;

    if state_dependent_xi == 0
        # Constant tilting function
        scale = 1.754;
        # scale = 1.32;
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        # Worrisome model
        alpha_z_tilde  = alpha_z_tilde_ex#-.0075
        kappa_tilde    = kappa_hat;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = ell_ex#0.055594409575544096

    elseif state_dependent_xi == 1
        # State-dependent tilting function (fixed kappa, alpha targets q)
        scale = 1.62
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        alpha_z_tilde  = alpha_z_tilde_ex#-.0075;
        kappa_tilde    =  .005
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = ell_ex#0.13852940062708508

    elseif state_dependent_xi == 2
        # State-dependent tilting function
        scale = 1.568
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        alpha_z_tilde  = alpha_z_tilde_ex#-.0075;
        kappa_tilde    = kappa_hat
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat + .1941
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat + .1941

        ell_star = ell_ex#0.18756641482672026

    end


elseif symmetric_returns == 0

    beta1_hat = 0.0;
    beta2_hat = 0.5;

    # (2) Technology
    phi2 = phi1 = phi_1cap;
    A2 = A1 = A_1cap;

    if state_dependent_xi == 0
        # Constant tilting function
        scale = 1.307
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat;

        # Worrisome model
        alpha_z_tilde  = alpha_z_tilde_ex#-.0075;
        kappa_tilde    = kappa_hat;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = ell_ex#0.026320287107624605

    elseif state_dependent_xi == 1
        # State-dependent tilting function (fixed kappa, alpha targets q)
        scale = 1.14
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat + .035; #.034;

        alpha_z_tilde  = alpha_z_tilde_ex#-.0075
        kappa_tilde    = .005;
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat;
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat

        ell_star = ell_ex#0.04226404306515605

    elseif state_dependent_xi == 2
        # State-dependent tilting function (fixed beta1, alpha targets q)
        scale = 1.27
        alpha_k2_hat = alpha_k1_hat = alpha_k_hat

        alpha_z_tilde  = alpha_z_tilde_ex#-.0075
        kappa_tilde    = kappa_hat
        alpha_k1_tilde = alpha_k1_hat
        beta1_tilde    = beta1_hat + .194 #.195
        alpha_k2_tilde = alpha_k2_hat
        beta2_tilde    = beta2_hat + .194 #.195

        ell_star = ell_ex#0.06678494013273199

    end

end

sigma_k1 = [.477*sqrt(scale),               .0,   .0];
sigma_k2 = [.0              , .477*sqrt(scale),   .0];


# (3) GRID
# For analysis
if compute_irfs == 1
    II, JJ = 7001, 501;     # number of r points, number of z points
    rmax = 4.;
    rmin = -rmax;
    zmax = .7;
    zmin = -zmax;
elseif compute_irfs == 0
    II, JJ = 1001, 201;
    rmax =  18.;
    rmin = -rmax       #-25.; #-rmax;
    zmax = 1.;
    zmin = -zmax;
end

# For the optimization (over ell)
II_opt, JJ_opt = 501, 201;     # number of r points, number of z points
rmax_opt = 18.;
rmin_opt = -rmax_opt;
zmax_opt = 1.2;
zmin_opt = -zmax_opt;


# (4) Iteration parameters
maxit = 500;        # maximum number of iterations in the HJB loop
crit  = 10e-6;      # criterion HJB loop
# Delta = 1000.;      # delta in HJB algorithm


# Initialize model objects -----------------------------------------------------
baseline = Baseline(alpha_z_hat, kappa_hat, sigma_z_1cap,
                    alpha_c_hat, beta_hat, sigma_c, delta);
baseline1 = Baseline(alpha_z_hat, kappa_hat, sigma_z,
                        alpha_k1_hat, beta1_hat, sigma_k1, delta);
baseline2 = Baseline(alpha_z_hat, kappa_hat, sigma_z,
                        alpha_k2_hat, beta2_hat, sigma_k2, delta);
technology = Technology(A_1cap, phi_1cap);
technology1 = Technology(A1, phi1);
technology2 = Technology(A2, phi2);
model = TwoCapitalEconomy(baseline1, baseline2, technology1, technology2);

worrisome = TwoCapitalWorrisome(alpha_z_tilde, kappa_tilde,
                                alpha_k1_tilde, beta1_tilde,
                                alpha_k2_tilde, beta2_tilde);
worrisome_noR = TwoCapitalWorrisome(alpha_z_hat, kappa_hat,
                                    alpha_k1_hat, beta1_hat,
                                    alpha_k2_hat, beta2_hat);

grid = Grid_rz(rmin, rmax, II, zmin, zmax, JJ);
grid_opt = Grid_rz(rmin_opt, rmax_opt, II_opt, zmin_opt, zmax_opt, JJ_opt);
params = FinDiffMethod(maxit, crit, Delta);

xi0, xi1, xi2 = tilting_function(worrisome, model);

if symmetric_returns == 0
    if state_dependent_xi == 0
        params.Delta = Delta;
    elseif state_dependent_xi == 1
        params.Delta = Delta;
    elseif state_dependent_xi == 2
        params.Delta = Delta;
    end
end

  Calibrated values: A:0.05  phi_1cap: 28.0  alpha_k : -1.2790328319261377  C/I : 0.5727486121839516  I/K : 0.03179147615369309


300

In [5]:
println(" (3) Compute value function WITH ROBUSTNESS")
A, V, val, d1_F, d2_F, d1_B, d2_B, h1_F, h2_F, hz_F, h1_B, h2_B, hz_B,
        mu_1_F, mu_1_B, mu_r_F, mu_r_B, mu_z, V0, rr, zz, pii, dr, dz =
        value_function_twocapitals(ell_star, rho, fraction, model, worrisome,
                                    grid, params, symmetric_returns);

 (3) Compute value function WITH ROBUSTNESS
rmax = 18.0, rmin = -18.0, rlength = 1001
zmax = 1.0, zmin = -1.0, zlength = 201
----------------------------------
Iteration = 1
Distance = 0.30849794108505413
v max = -1.3029841589245825
v min = -1.9110044304703626
d1 max = 0.031791408190397795
d1 min = 0.031791388018299754
d2 max = 0.03179140847617034
d2 min = 0.03179138644625998
h1 max = 0.007883939080485569
h1 min = 0.0024306796433816507
h2 max = 0.007883939080485569
h2 min = 0.0024306796433816507
hz max = 0.007812500000000111
hz min = 0.007812499999999556
----------------------------------
----------------------------------
Iteration = 2
Distance = 0.028372158232104816
v max = -1.3106248946663819
v min = -1.9185802250355548
d1 max = 0.032867641590059504
d1 min = 0.03070542685284094
d2 max = 0.03286763443681707
d2 min = 0.030705438789140026
h1 max = 0.005640228243266833
h1 min = 0.0006676318121324689
h2 max = 0.007883932230418816
h2 min = 5.135457212221425e-5
hz max = 0.00781247799511108

In [6]:
one_pii = 1 .- pii
# Define Policies object
policies  = PolicyFunctions(d1_F, d2_F, d1_B, d2_B,
                            -h1_F/ell_star, -h2_F/ell_star, -hz_F/ell_star,
                            -h1_B/ell_star, -h2_B/ell_star, -hz_B/ell_star);

# Construct drift terms under the baseline
mu_1 = (mu_1_F + mu_1_B)/2.;
mu_r = (mu_r_F + mu_r_B)/2.;
# ... under the worst-case model
h1_dist = (policies.h1_F + policies.h1_B)/2.;
h2_dist = (policies.h2_F + policies.h2_B)/2.;
hz_dist = (policies.hz_F + policies.hz_B)/2.;

# local uncertainty prices
h1, h2, hz = -h1_dist, -h2_dist, -hz_dist;

d1 = (policies.d1_F + policies.d1_B)/2;
d2 = (policies.d2_F + policies.d2_B)/2;
cons     = one_pii .* (model.t1.A .- d1) + pii .* (model.t2.A .- d2);

In [36]:
# using Tables
# CSV.write(filename_ell*"d1.csv",  Tables.table(d1), writeheader=false)
# CSV.write(filename_ell*"d2.csv",  Tables.table(d2), writeheader=false)
# CSV.write(filename_ell*"h1.csv",  Tables.table(h1), writeheader=false)
# CSV.write(filename_ell*"h2.csv",  Tables.table(h2), writeheader=false)
# CSV.write(filename_ell*"hz.csv",  Tables.table(hz), writeheader=false)
# CSV.write(filename_ell*"V.csv",  Tables.table(V), writeheader=false)
# CSV.write(filename_ell*"cons.csv",  Tables.table(cons), writeheader=false)

results = Dict("delta" => delta,
# Two capital stocks
"sigma_k1" => sigma_k1, "sigma_k2" => sigma_k2,
"sigma_z" =>  sigma_z, "A1" => A1, "A2" => A2, "phi1" => phi1, "phi2" => phi2,
"I" => II, "J" => JJ,
"rmax" => rmax, "rmin" => rmin, "zmax" => zmax, "zmin" => zmin,
"rr" => rr, "zz" => zz, "pii" => pii, "dr" => dr, "dz" => dz,
"maxit" => maxit, "crit" => crit, "Delta" => Delta,

"V0" => V0, "V" => V, "val" => val, "ell_star" => ell_star,

"d1" => d1, "d2" => d2,

"h1" => h1, "h2" => h2, "hz" => hz,

"cons" => cons,

"A_1cap" => A_1cap, "phi_1cap" => phi_1cap)

filename_ell = "./output/"*dataname*"/gamma_"*string(gamma)*"_rho_"*string(rho)*"/"
isdir(filename_ell) || mkpath(filename_ell)
npzwrite(filename_ell*filename, results)


In [30]:
filename_ell

"./output/local/"